In [62]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
import random
import pandas as pd
from multiprocessing.dummy import Pool

In [63]:
#淘宝和天猫的 搜索页面
init_searchurl_taobao ='https://s.taobao.com/search?'
#淘宝和天猫的商品链接
init_itemurl_tmall = 'https://detail.tmall.com/item.htm?id={}'
init_itemurl_taobao = 'https://item.taobao.com/item.htm?id={}'

headers={
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36'
        }
#淘宝搜索页面
init_searchurl_taobao ='https://s.taobao.com/search?'

In [64]:
def getitemurls_taobao(target,sort='sale-desc',maxpage=1):
    itemurls = {}
    itemurl_taobao = []
    itemurl_tmall = []
    for page in range(0,maxpage):
        params = {
                    'q' : target,
                    'sort' : sort,
                    's' : str(44*(page))
        }
        res = requests.get(init_searchurl_taobao,params=params,headers=headers)
        ids = re.findall('"nid":"(.*?)","category"',res.text,re.S)
        urls = re.findall('"detail_url":"//(.*?)","view_price"',res.text,re.S)
#        print(len(ids))
#        print(len(urls))
        if len(ids) == 0:
            print('被干了这次不算')
        info = dict(ids = ids , urls=urls)
        for i in range(0,len(ids)):
        #由于淘宝页面返回的URL是带//OOOC3这种乱码的，所以url只是看一下到底是淘宝还是天猫，并不作为真正的url传回去
        #返回的URL是用init_itemurl拼出来的
            if 'tmall' in info['urls'][i]:
                itemurl_tmall.append(init_itemurl_tmall.format(info['ids'][i]))
            elif 'taobao' in info['urls'][i]:
                itemurl_taobao.append(init_itemurl_taobao.format(info['ids'][i]))  
#        print("稍等一下")
        time.sleep(random.randint(0,1))
    itemurls['tmall'] = itemurl_tmall
    itemurls['taobao'] = itemurl_taobao
    print('共有商品%d个，其中淘宝%d个，天猫%d个'%((len(itemurl_tmall)+len(itemurl_taobao)),len(itemurl_taobao),len(itemurl_tmall)))
    return itemurls

In [65]:
#获取淘宝的商品信息
def getitemInfo_taobao(itemUrl):
    res = openlink(itemUrl)
    print("获取商品: "+ res.url)
    selector = etree.HTML(res.text)
    title = selector.xpath("//h3[@class='tb-main-title']/@data-title")[0]
    price = selector.xpath("//em[@class = 'tb-rmb-num']/text()")[0] # 淘宝
    itemID = re.findall('id=(\d+)',res.url)[0]
    itemDetail = "".join(selector.xpath('//div[@class="attributes"]/ul/li/text()'))
    itemInfo = pd.DataFrame(dict(title=title,
                                 price=price,
                                itemID=itemID,
                                itemDetail=[itemDetail],
                                itemLink =res.url),columns=['itemID','title' , 'price' ,'itemDetail','itemLink'])
    return itemInfo

In [66]:
# 获取天猫的商品信息
def getitemInfo_tmall(itemUrl):
        #res = requests.get(itemUrl,headers=headers)
        res = openlink(itemUrl)
        print("获取商品: "+ res.url)
        selector = etree.HTML(res.text)    
        title = selector.xpath("//div[@class='tb-detail-hd']/h1/text()")[0].replace(' ','').replace('\t','').replace('\r','').replace('\n','')
        price = re.findall('"defaultItemPrice":"(.*?)"',res.text)[0]
        itemID = re.findall('id=(\d+)',res.url)[0]
        itemDetail = "".join(selector.xpath('//div[@class="attributes"]/div/ul/li/text()'))
        itemInfo = pd.DataFrame(dict(title=title,
                                     price=price,
                                    itemID=itemID,
                                    itemDetail=[itemDetail],
                                    itemLink =res.url),columns=['itemID','title' , 'price' ,'itemDetail','itemLink'])
        return itemInfo

In [67]:
def openlink(link):
    maxTryNum = 10
    for tries in range(0,maxTryNum):
        try:
            res = requests.get(link, headers = headers)
            return res
        except:
            if tries < (maxTryNum - 1):
                print("啊呀")
                continue
            else:
                print("Has tried %d times to access url %s, all failed!", maxNum, link)
                break

In [68]:
target = 'ADAS'
itemUrl = getitemurls_taobao(target,maxpage=5)

共有商品220个，其中淘宝172个，天猫48个


In [69]:
texturls = itemUrl

In [43]:
info_taobao =[]
info_tmall = []

pool = Pool(8)
time3 =time.time()
info_taobao = pool.map(getitemInfo_taobao,texturls['taobao'])
info_tmall = pool.map(getitemInfo_tmall,texturls['tmall'])
pool.close()
pool.join()

time4 =time.time()
print(str(time4-time3))

获取商品: https://item.taobao.com/item.htm?id=557348813127
获取商品: https://item.taobao.com/item.htm?id=564811926741
获取商品: https://item.taobao.com/item.htm?id=572104625658
获取商品: https://item.taobao.com/item.htm?id=572940460076
获取商品: https://item.taobao.com/item.htm?id=571591542201
获取商品: https://item.taobao.com/item.htm?id=566098788792
获取商品: https://item.taobao.com/item.htm?id=566953407543
获取商品: https://item.taobao.com/item.htm?id=554381472455
获取商品: https://item.taobao.com/item.htm?id=556671120518
获取商品: https://item.taobao.com/item.htm?id=563540476428
获取商品: https://item.taobao.com/item.htm?id=565160457366
获取商品: https://item.taobao.com/item.htm?id=571431937804
获取商品: https://item.taobao.com/item.htm?id=567927751814
获取商品: https://item.taobao.com/item.htm?id=562451513433
获取商品: https://item.taobao.com/item.htm?id=572389229748
获取商品: https://item.taobao.com/item.htm?id=564445275974
获取商品: https://item.taobao.com/item.htm?id=571418753656
获取商品: https://item.taobao.com/item.htm?id=563839483640
获取商品: http

啊呀
获取商品: https://item.taobao.com/item.htm?id=569026835328
获取商品: https://item.taobao.com/item.htm?id=573240542512
啊呀
获取商品: https://item.taobao.com/item.htm?id=566054155063
获取商品: https://item.taobao.com/item.htm?id=544551798706
获取商品: https://item.taobao.com/item.htm?id=571661641713
啊呀
获取商品: https://item.taobao.com/item.htm?id=536589759911
啊呀
获取商品: https://item.taobao.com/item.htm?id=560243141867
获取商品: https://item.taobao.com/item.htm?id=557104767598
获取商品: https://item.taobao.com/item.htm?id=570037521517
获取商品: https://item.taobao.com/item.htm?id=561401643563
啊呀
啊呀
获取商品: https://item.taobao.com/item.htm?id=570813534330
啊呀
获取商品: https://item.taobao.com/item.htm?id=573375381773
获取商品: https://item.taobao.com/item.htm?id=565339504405
获取商品: https://item.taobao.com/item.htm?id=562798694187
获取商品: https://item.taobao.com/item.htm?id=556865249692
获取商品: https://item.taobao.com/item.htm?id=571182040644
获取商品: https://item.taobao.com/item.htm?id=573793773634
啊呀
获取商品: https://item.taobao.com/item.htm?id

In [60]:
tb = pd.concat(info_taobao,ignore_index=True)
tm = pd.concat(info_tmall,ignore_index=True)
df = pd.merge(tb,tm,how='outer')
df.to_csv(str('output/'+target+'-info-' +str(time.strftime("%Y-%m-%d"))+'.csv'))

In [61]:
str(time.strftime("%Y-%m-%d"))

'2018-08-07'